stuff we did

- visualization
- anallysis
- cleaning

- faceted line graphs can help
- sample representative communities instead of using all is fine as long as it is accurate

In [ ]:
import pandas as pd



In [ ]:
# TODO clean population estimates so it only includes the communities that we want
# TODO fix tables 2b-2 and 2b
# TODO start working on faceted graph
# TODO for population before graph, make as ranges

Ok so here is the shit that I am going to do for my part, please listen to the checklist ANALYSIS
- make **Faceted Graph** for table 8 (and 6 now)
    - write observations if any
    - **MAINLY, WHAT RELATION BETWEEN MONEY MONEY AND VACANCY IS THERE??**

- <u>Sampled Towns</u> --> how might these be useful?
    - s


New changes
- removed commas in thousands from Table 8 --> can read as numerical values now
- removed "Overall Vacancy Rates" from Table 8, since this is covered in table 6 and is more accurate there, just need the weighted rents between years in table 8\
- added "population" which we want to analyze how town population sizes might also be related to costs and shit
    - add to faceted graph as an outer variable